## Imports

In [ ]:
# !pip install scikit-video
import cv2
import numpy as np
import pandas as pd
from google.colab.patches import cv2_imshow
import skvideo.io
import skvideo.measure

## Generate Video Outputs
Our method: both false

Naive downsampling (Every Nth frames): downsample true

Every Nth frames with random offset: both true

In [ ]:
simulate_downsampling = False
simulate_randomchunkselection = False

mse_csv_filepath = "/content/mark_back_and_forth_square_mse_sum_500_0.98.csv"
original_video_filepath = "/content/mark_rolling_on_chair.mp4"
output_file_name = "output.mp4"
output_fps = 30

In [ ]:
mse = np.array(pd.read_csv(mse_csv_filepath))[:,1]
mse = mse.reshape(8,14,14)
mse = mse - mse.min()
mse = mse / mse.mean()
mse = mse / 5

if simulate_downsampling:
  regdown = np.zeros(mse.shape)
  regdown.fill(mse.mean())
  mse = regdown

# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
cap = cv2.VideoCapture(original_video_filepath)

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
 
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(frame_width, frame_height, length)

prev_frame = np.zeros((frame_height, frame_width, 3), dtype=np.uint8)
prev_increment = np.ones((14,14))
if (simulate_randomchunkselection):
  prev_increment = np.random.rand(14,14)
  prev_increment += 1
 
out = cv2.VideoWriter('output_file_name',cv2.VideoWriter_fourcc(*'mp4v'), output_fps, (frame_width, frame_height))
# Read until video is completed
count = 0
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
    pass

    next_frame = np.copy(prev_frame)
    losses = mse[8 * count//length, :, :]
    prev_increment += losses

    for i in range(14):
      left = i * frame_width // 14
      right = (i+1) * frame_width // 14
      for j in range(14):
        top = j * frame_height // 14
        bottom = (j+1) * frame_height // 14

        if prev_increment[i][j] >= 1:
          prev_increment[i][j] += -1
          next_frame[left:right, top:bottom] = frame[left:right, top:bottom]
          prev_frame[left:right, top:bottom] = frame[left:right, top:bottom] 
      
    out.write(next_frame)

    count += 1
  # Break the loop
  else: 
    break
 
# When everything done, release the video capture object
cap.release()
out.release()
 
# Closes all the frames
cv2.destroyAllWindows()

## Video Metrics
put paths below

In [ ]:
original  = skvideo.io.vread("/content/mark_rolling_on_chair.mp4")
ours      = skvideo.io.vread("/content/mark_out5.mp4")
naive     = skvideo.io.vread("/content/mark_out5downsample.mp4")
random    = skvideo.io.vread("/content/mark_out5random.mp4")

original_grey =skvideo.io.vread("/content/mark_rolling_on_chair.mp4", as_grey = True)
ours_grey     =skvideo.io.vread("/content/mark_out5.mp4", as_grey = True)
naive_grey    =skvideo.io.vread("/content/mark_out5downsample.mp4", as_grey = True)
random_grey   =skvideo.io.vread("/content/mark_out5random.mp4", as_grey = True)

In [ ]:
toCompare = [ours, naive, random]
toCompare_grey = [ours_grey, naive_grey, random_grey]
for video in toCompare:
  mse = (np.square(original - video)).mean()
  print("MSE: ", mse)

for video in toCompare_grey:
  psnr = skvideo.measure.psnr(original_grey, video)
  print("PSNR mean: ", psnr.mean())

for video in toCompare_grey:
  ssim = skvideo.measure.ssim(original_grey, video)
  print(ssim.mean())

for video in toCompare_grey:
  strred, final_after, final_before = skvideo.measure.strred(original_grey, video)
  print("ST-RRED: ", final_after, "ST-RRED SSN: ", final_before)

66.25909085602545
